In [ ]:
import pandas as pd
import folium
import math
DS_DMT = pd.read_csv(r"C:\Khue\H9_Solar_Power_Forecasting\data\raw\thongtintinh_farm.csv")
DS_DMT


In [43]:
# Loại bỏ các dòng có giá trị null ở cột 'kinh độ và vĩ độ
DS_DMT = DS_DMT.dropna(subset=['KINHDO', 'VIDO'])
DS_DMT = DS_DMT.loc[DS_DMT['Loại hình']=="Mặt trời"]

In [45]:
# Tính trung tâm của các điểm để khởi tạo bản đồ
center_lat = (30 + 0) / 2  # Center latitude between 30°N and 0°N
center_lon = (80 + 115) / 2  # Center longitude between 80°E and 115°E

# Tạo bản đồ Folium với vị trí trung tâm
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

folium.Rectangle(
    bounds=[[30, 80], [0, 115]],
    color='red',
    fill=False,
    weight=2
).add_to(m)

# Hàm tính delta kinh độ (tính theo độ) cho bán cạnh 10 km tại mỗi điểm
def compute_delta_lon(lat, half_side_km=25):
    return half_side_km / (111 * math.cos(math.radians(lat)))

# Tính delta vĩ độ cho bán cạnh 10 km (vì 1° vĩ độ ≈ 111 km)
delta_lat = 25 / 111

# Vẽ marker và hình vuông cho mỗi nhà máy điện mặt trời
for idx, row in DS_DMT.iterrows():
    lat = row['VIDO']
    lon = row['KINHDO']
    
    # Tính delta kinh độ tại vị trí hiện tại
    delta_lon = compute_delta_lon(lat)
    
    # Xác định biên của hình vuông (vùng bao gồm 20 km mỗi cạnh)
    sw = [lat - delta_lat, lon - delta_lon]  # góc Tây Nam
    ne = [lat + delta_lat, lon + delta_lon]  # góc Đông Bắc
    
    # Vẽ hình vuông (Rectangle)
    folium.Rectangle(
        bounds=[sw, ne],
        color='blue',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)
    
    # Nếu cột 'name' có giá trị null, sử dụng tên mặc định
    plant_name = row['TEN_NM']
    
    # Vẽ marker tại vị trí của nhà máy
    folium.Marker(
        location=[lat, lon],
        popup=plant_name,
    ).add_to(m)
    

# Fit the map bounds to the rectangle
m.fit_bounds([[30, 80], [0, 115]])

m    


In [ ]:
DS_DMT[]

In [ ]:
import pandas as pd
from PIL import Image, ImageDraw
import math

# Bước 1: Đọc dữ liệu CSV
df = pd.read_csv(r"C:\Khue\H9_Solar_Power_Forecasting\data\raw\thongtintinh_farm.csv")
df = df.dropna(subset=["VIDO", "KINHDO"])  # loại bỏ dòng thiếu toạ độ
df = df['TEN_NM']=="MT Krông Pa"
# Bước 2: Mở ảnh
img = Image.open(r"C:\Khue\H9_Solar_Power_Forecasting\references\20250101\b03_0000.jpg")
width, height = img.size

# Bước 3: Định nghĩa bounding box (theo giả định)
#   Trên-trái  = (lon=80,  lat=30)
#   Dưới-phải = (lon=115, lat=0)
min_lon, max_lon = 80, 115
max_lat, min_lat = 30, 0

# Bước 4: Hàm chuyển (lat, lon) -> (x, y) pixel
def latlon_to_pixel(lat, lon,
                    min_lon=min_lon, max_lon=max_lon,
                    min_lat=min_lat, max_lat=max_lat,
                    img_width=width, img_height=height):
    """
    Giả sử ảnh trải dài từ (min_lon, max_lat) đến (max_lon, min_lat).
    Trả về toạ độ pixel (x, y) trên ảnh.
    """
    # Tính toạ độ x (theo kinh độ)
    x = (lon - min_lon) / (max_lon - min_lon) * img_width

    # Tính toạ độ y (theo vĩ độ, lưu ý vĩ độ giảm từ bắc xuống nam)
    y = (max_lat - lat) / (max_lat - min_lat) * img_height
    return x, y

# Bước 5: Chuẩn bị công cụ vẽ
draw = ImageDraw.Draw(img)

# Bước 6: Định nghĩa bán cạnh (nửa chiều dài cạnh ô vuông)
#         30 km cạnh -> half_side_km = 15 km
half_side_km = 15

# Tính delta vĩ độ (dựa trên 1° vĩ độ ~ 111 km)
def compute_delta_lat(half_side_km):
    return half_side_km / 111

# Tính delta kinh độ (thay đổi theo cos(vĩ độ))
def compute_delta_lon(lat, half_side_km):
    return half_side_km / (111 * math.cos(math.radians(lat)))

# Bước 7: Vẽ cho từng điểm
for idx, row in df.iterrows():
    lat_center, lon_center = row['VIDO'], row['KINHDO']
    
    # Toạ độ pixel trung tâm
    x_center, y_center = latlon_to_pixel(lat_center, lon_center)
    
    # Tính “bán cạnh” theo vĩ độ và kinh độ
    d_lat = compute_delta_lat(half_side_km)             # ~ 0.135° (nếu lat không quá lớn)
    d_lon = compute_delta_lon(lat_center, half_side_km) # tuỳ thuộc vĩ độ
    
    # Tính toạ độ (lat, lon) các góc
    lat_north = lat_center + d_lat
    lat_south = lat_center - d_lat
    lon_west  = lon_center - d_lon
    lon_east  = lon_center + d_lon
    
    # Đổi các góc sang pixel
    x_nw, y_nw = latlon_to_pixel(lat_north, lon_west)
    x_ne, y_ne = latlon_to_pixel(lat_north, lon_east)
    x_sw, y_sw = latlon_to_pixel(lat_south, lon_west)
    x_se, y_se = latlon_to_pixel(lat_south, lon_east)
    
    # Lấy bounding box (x_min, y_min, x_max, y_max) để vẽ rectangle
    x_min = min(x_nw, x_ne, x_sw, x_se)
    y_min = min(y_nw, y_ne, y_sw, y_se)
    x_max = max(x_nw, x_ne, x_sw, x_se)
    y_max = max(y_nw, y_ne, y_sw, y_se)
    
    # Vẽ hình vuông (Rectangle) viền màu xanh
    draw.rectangle([x_min, y_min, x_max, y_max], outline='blue', width=2)
    
    

# Bước 8: Lưu ảnh kết quả
output_path = "b03_with_squares.jpg"
img.save(output_path)
print(f"Đã lưu ảnh kết quả: {output_path}")


In [71]:
import pandas as pd
from PIL import Image, ImageDraw
import math

# 1) Đọc CSV
df = pd.read_csv(r"C:\Khue\H9_Solar_Power_Forecasting\data\raw\thongtintinh_farm.csv")
df = df.dropna(subset=["VIDO", "KINHDO"])  # Bỏ dòng thiếu lat/lon

# 2) Lọc dữ liệu cho MT Krông Pa
mt_krong_pa = df[df['TEN_NM'] == 'MT Solarpark 1']

# 3) Mở ảnh
img = Image.open(r"C:\Khue\H9_Solar_Power_Forecasting\notebooks\arm_20250101_0000.jpg")
width, height = img.size

# 4) Định nghĩa bounding box ảnh (theo giả định)
#    Góc trên-trái:  (80°E, 30°N)
#    Góc dưới-phải: (115°E, 0°N)
min_lon, max_lon = 80, 115
max_lat, min_lat = 30, 0

# 5) Hàm chuyển (lat, lon) -> (x, y) pixel
def latlon_to_pixel(lat, lon,
                    min_lon=min_lon, max_lon=max_lon,
                    min_lat=min_lat, max_lat=max_lat,
                    img_width=width, img_height=height):
    # Kinh độ nội suy theo chiều ngang
    x = (lon - min_lon) / (max_lon - min_lon) * img_width
    # Vĩ độ nội suy theo chiều dọc (lưu ý: y tăng từ trên xuống)
    y = (max_lat - lat) / (max_lat - min_lat) * img_height
    return x, y

# 6) Bán cạnh (nếu muốn ô vuông cạnh 30 km)
half_side_km = 50

def compute_delta_lat(half_side_km):
    return half_side_km / 111

def compute_delta_lon(lat, half_side_km):
    return half_side_km / (111 * math.cos(math.radians(lat)))


# 8) Định nghĩa bán cạnh (nếu muốn ô vuông cạnh 30 km)
for idx, row in mt_krong_pa.iterrows():
    lat_center = row['VIDO']
    lon_center = row['KINHDO']
    
    x_center, y_center = latlon_to_pixel(lat_center, lon_center)
    
    d_lat = compute_delta_lat(half_side_km)
    d_lon = compute_delta_lon(lat_center, half_side_km)
    
    lat_north = lat_center + d_lat
    lat_south = lat_center - d_lat
    lon_west  = lon_center - d_lon
    lon_east  = lon_center + d_lon
    
    x_nw, y_nw = latlon_to_pixel(lat_north, lon_west)
    x_ne, y_ne = latlon_to_pixel(lat_north, lon_east)
    x_sw, y_sw = latlon_to_pixel(lat_south, lon_west)
    x_se, y_se = latlon_to_pixel(lat_south, lon_east)
    
    x_min = min(x_nw, x_ne, x_sw, x_se)
    y_min = min(y_nw, y_ne, y_sw, y_se)
    x_max = max(x_nw, x_ne, x_sw, x_se)
    y_max = max(y_nw, y_ne, y_sw, y_se)
    
    # # Vẽ ô vuông
    # draw.rectangle([x_min, y_min, x_max, y_max])
    
# 9) Hàm crop vuông, có cơ chế "dời" (shift) nếu tràn
def crop_square(img, x_min, y_min, x_max, y_max):
    """
    Crop ảnh thành hình vuông bao trọn vùng [x_min, y_min, x_max, y_max] với kích thước lớn hơn 10 lần.
    Nếu vùng crop tràn ra ngoài, sẽ dời trong phạm vi còn trống.
    """
    W, H = img.size
    bbox_w = x_max - x_min
    bbox_h = y_max - y_min
    
    side = max(bbox_w, bbox_h)  # cạnh hình vuông
    cx = (x_min + x_max) / 2
    cy = (y_min + y_max) / 2
    
    side = int(math.ceil(side))  # làm tròn lên để chắc chắn không cắt mất
    if side > W or side > H:
        print("Vùng crop yêu cầu lớn hơn kích thước ảnh! Giữ nguyên ảnh.")
        return img
    
    # Tính toạ độ ban đầu
    crop_xmin = int(cx - side/2)
    crop_ymin = int(cy - side/2)
    crop_xmax = crop_xmin + side
    crop_ymax = crop_ymin + side
    
    # Dời nếu tràn
    if crop_xmin < 0:
        shift = -crop_xmin
        crop_xmin = 0
        crop_xmax += shift
    if crop_xmax > W:
        shift = crop_xmax - W
        crop_xmax = W
        crop_xmin -= shift
    if crop_ymin < 0:
        shift = -crop_ymin
        crop_ymin = 0
        crop_ymax += shift
    if crop_ymax > H:
        shift = crop_ymax - H
        crop_ymax = H
        crop_ymin -= shift
    
    # Đảm bảo vẫn vuông (nếu có lệch 1 pixel do làm tròn, ta điều chỉnh)
    final_w = crop_xmax - crop_xmin
    final_h = crop_ymax - crop_ymin
    if final_w != final_h:
        side2 = min(final_w, final_h)
        crop_xmax = crop_xmin + side2
        crop_ymax = crop_ymin + side2
    
    return img.crop((crop_xmin, crop_ymin, crop_xmax, crop_ymax))

# 10) Crop ảnh lớn hơn 10 lần ô vuông của MT Krông Pa
cropped_img = crop_square(img, x_min, y_min, x_max, y_max)

# Phóng to ảnh lên 10 lần kích thước ban đầu
resized_img = cropped_img.resize((cropped_img.size[0]*25, cropped_img.size[1]*25), Image.LANCZOS)
cropped_img = resized_img
# 11) Lưu ảnh
out_path = "b03_with_squares_krong_pa_cropped_large00.jpg"
cropped_img.save(out_path)

In [ ]:
import pandas as pd
from PIL import Image
import math
import os

def crop_and_resize_image(image_path, plant_name, scale_factor=10, output_root=r"X:\Sky-image\Processed"):
    """
    Cắt ảnh bao trọn nhà máy được chọn, sau đó mở rộng ảnh theo scale_factor và lưu vào thư mục có cấu trúc tương tự thư mục gốc.

    Parameters:
    - image_path (str): Đường dẫn ảnh gốc.
    - plant_name (str): Tên nhà máy cần xử lý.
    - scale_factor (int): Hệ số phóng to ảnh (mặc định là 10 lần).
    - output_root (str): Thư mục gốc để lưu ảnh đã xử lý.

    Returns:
    - str: Đường dẫn ảnh sau khi xử lý.
    """
    
    # 1) Đọc dữ liệu nhà máy
    csv_path = r"C:\Khue\H9_Solar_Power_Forecasting\data\raw\thongtintinh_farm.csv"
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=["VIDO", "KINHDO"])  # Bỏ dòng thiếu lat/lon

    # 2) Lọc dữ liệu theo tên nhà máy
    plant_data = df[df['TEN_NM'] == plant_name]
    
    if plant_data.empty:
        print(f"Không tìm thấy nhà máy '{plant_name}' trong dữ liệu.")
        return None

    # 3) Mở ảnh
    img = Image.open(image_path)
    width, height = img.size

    # 4) Xác định khung tọa độ
    min_lon, max_lon = 80, 115
    max_lat, min_lat = 30, 0

    # 5) Chuyển đổi tọa độ lat/lon sang pixel
    def latlon_to_pixel(lat, lon):
        x = (lon - min_lon) / (max_lon - min_lon) * width
        y = (max_lat - lat) / (max_lat - min_lat) * height
        return x, y

    # 6) Hàm tính khoảng cách theo lat/lon
    def compute_delta_lat(km):
        return km / 111

    def compute_delta_lon(lat, km):
        return km / (111 * math.cos(math.radians(lat)))

    # 7) Xác định vùng cần crop
    half_side_km = 50  # 50km bán kính từ trung tâm nhà máy

    for _, row in plant_data.iterrows():
        lat_center, lon_center = row['VIDO'], row['KINHDO']
        x_center, y_center = latlon_to_pixel(lat_center, lon_center)

        d_lat = compute_delta_lat(half_side_km)
        d_lon = compute_delta_lon(lat_center, half_side_km)

        lat_north, lat_south = lat_center + d_lat, lat_center - d_lat
        lon_west, lon_east = lon_center - d_lon, lon_center + d_lon

        x_nw, y_nw = latlon_to_pixel(lat_north, lon_west)
        x_se, y_se = latlon_to_pixel(lat_south, lon_east)

        x_min, y_min = min(x_nw, x_se), min(y_nw, y_se)
        x_max, y_max = max(x_nw, x_se), max(y_nw, y_se)

    # 8) Hàm crop ảnh theo vùng
    def crop_square(img, x_min, y_min, x_max, y_max, scale_factor):
        W, H = img.size
        bbox_w, bbox_h = x_max - x_min, y_max - y_min
        side = max(bbox_w, bbox_h) * scale_factor

        cx, cy = (x_min + x_max) / 2, (y_min + y_max) / 2
        crop_xmin, crop_ymin = int(cx - side / 2), int(cy - side / 2)
        crop_xmax, crop_ymax = crop_xmin + int(side), crop_ymin + int(side)

        # Dời nếu tràn viền
        if crop_xmin < 0:
            crop_xmax -= crop_xmin
            crop_xmin = 0
        if crop_xmax > W:
            crop_xmin -= (crop_xmax - W)
            crop_xmax = W
        if crop_ymin < 0:
            crop_ymax -= crop_ymin
            crop_ymin = 0
        if crop_ymax > H:
            crop_ymin -= (crop_ymax - H)
            crop_ymax = H

        return img.crop((crop_xmin, crop_ymin, crop_xmax, crop_ymax))

    # 9) Cắt ảnh và phóng to
    cropped_img = crop_square(img, x_min, y_min, x_max, y_max, scale_factor)
    resized_img = cropped_img.resize((cropped_img.size[0] * 25, cropped_img.size[1] * 25), Image.LANCZOS)

    # 10) **Tạo đường dẫn lưu ảnh theo cấu trúc đầu vào**
    input_root = r"X:\Sky-image\Data"  # Thư mục gốc của dữ liệu
    relative_path = os.path.relpath(image_path, input_root)  # Lấy đường dẫn tương đối
    output_path = os.path.join(output_root, relative_path)  # Đường dẫn file đầu ra

    # Tạo thư mục đích nếu chưa tồn tại
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    # 11) Lưu ảnh
    resized_img.save(output_path)
    print(f"✅ Ảnh đã lưu tại: {output_path}")

    return output_path
